In [273]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
import itertools

In [274]:
df = pd.read_excel("fixed.xlsx", engine="openpyxl")

c:\Users\Beste\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\Beste\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


In [275]:
df.rename(columns= {"Column1" : "Rank"}, inplace= True)
df.rename(columns= {"Column2" : "Title"}, inplace= True)
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [276]:
df["Lyrics"][0]

"3 There's a summer place_x000D_\nWhere it may rain or storm_x000D_\nYet I'm safe and warm_x000D_\nFor within that summer place_x000D_\nYour arms reach out to me_x000D_\nAnd my heart is free from all care_x000D_\nFor it knows_x000D_\n_x000D_\nThere are no gloomy skies_x000D_\nWhen seen through the eyes_x000D_\nOf those who are blessed with love_x000D_\n_x000D_\nAnd the sweet secret of_x000D_\nA summer place_x000D_\nIs that it's anywhere_x000D_\nWhen two people share_x000D_\nAll their hopes_x000D_\nAll their dreams_x000D_\nAll their love_x000D_\n_x000D_\nAnd the sweet secret of a summer place_x000D_\nIs that it's anywhere_x000D_\nWhen two people share_x000D_\nAll their hopes_x000D_\nAll their dreams, all their love"

In [277]:
# Strings to be replaced
old_string = '_x000D_'
new_string = ' '

# Replace the string in the DataFrame
df = df.map(lambda x: x.replace(old_string, new_string) if isinstance(x, str) else x)
df['Lyrics'] = df['Lyrics'].astype(str).str.replace('\n', ' ')


In [278]:
type(df["Year"][0])

numpy.int64

In [279]:
# Define the corpus
corpus = df["Lyrics"].tolist()
print(len(corpus))
type(corpus)

6400


list

In [280]:
# Tokenize
lyric_corpus_tokenized = []
tokenizer = RegexpTokenizer(r'\w+')
for lyric in corpus:
    tokenized_lyric = tokenizer.tokenize(lyric.lower())
    lyric_corpus_tokenized.append(tokenized_lyric)

print(len(lyric_corpus_tokenized))

6400


In [281]:
# filter short words
for s, song in enumerate(lyric_corpus_tokenized):
    filtered_song = []
    for token in song:
        if len(token) > 2:
            filtered_song.append(token)
    lyric_corpus_tokenized[s] = filtered_song

In [282]:
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Lemmatize the words
for s, song in enumerate(lyric_corpus_tokenized):
    lemmatized_tokens = []
    for token in song:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))
    lyric_corpus_tokenized[s] = lemmatized_tokens

In [283]:
from nltk.corpus import stopwords

# Filter stopwords
stop_words = stopwords.words("english")
new_stop_words = [
    'ooh', 'yeah', 'hey', 'whoa', 'woah', 'ohh', 'mmm', 'oooh', 'yah', 'yeh', 'hmm',
    'deh', 'doh', 'jah', 'let', 'ah', 'uhh', 'ha', 'eh', 'wow', 'yay', 'oops', 'ouch',
    'phew', 'ugh', 'yup', 'nope', 'okay', 'ok', 'nah', 'duh', 'aww', 'tsk', 'shh',
    'whee', 'yay', 'huh', 'bro', 'dude', 'sis', 'man', 'buddy', 'pal', 'gal', 'like',
    'literally', 'basically', 'actually', 'totally', 'seriously', 'just', 'really',
    'very', 'so', 'well', 'you know', 'I mean', 'kind of', 'sort of', 'kind', 'sort',
    'maybe', 'perhaps', 'aint', 'gonna', 'wanna', 'gotta', 'lemme', 'dunno', 'kinda',
    'sorta', 'whatcha', 'uh-huh', 'huh-uh', 'woohoo', 'be', 'have', 'do', 'say', 'get',
    'make', 'go', 'know', 'take', 'see', 'come', 'think', 'look', 'want', 'give', 'use',
    'find', 'tell', 'ask', 'seem', 'feel', 'try', 'leave', 'call', 'is', 'are',
    'was', 'were', 'been', 'am', 'has', 'had', 'does', 'did', 'says', 'gets', 'makes',
    'goes', 'knows', 'takes', 'sees', 'comes', 'thinks', 'looks', 'wants', 'gives', 'uses',
    'finds', 'tells', 'asks', 'works', 'seems', 'tries', 'leaves', "said", "new", "got", "ayy", "gon", "doo", 
    "wa", "away", "one", "two", "put", "ever", "never", "cause", "time", "every", "always"
]
stop_words.extend(new_stop_words)

for s, song in enumerate(lyric_corpus_tokenized):
    filtered_text = []
    for token in song:
        if token not in stop_words:
            filtered_text.append(token)
    lyric_corpus_tokenized[s] = filtered_text

In [284]:
from gensim.corpora import Dictionary

# Define the dictionary
dictionary = Dictionary(lyric_corpus_tokenized)
dictionary.filter_extremes(no_below = 20, no_above=0.5, keep_n=50000)
print(dictionary)

Dictionary<2609 unique tokens: ['anywhere', 'arm', 'blessed', 'care', 'dream']...>


In [285]:
from gensim.corpora import MmCorpus

# Create the dictionary for gensim
gensim_corpus = [dictionary.doc2bow(song) for song in lyric_corpus_tokenized]
temp = dictionary[0]
id2word = dictionary.id2token

In [286]:
# Define the parameters
chunksize = 500
passes = 20
iterations = 50
num_topics = 4

In [287]:
from gensim.models import LdaModel

# Train the model
lda_model = LdaModel(
    corpus = gensim_corpus,
    id2word = id2word,
    chunksize = chunksize,
    alpha= "auto",
    eta = "auto", 
    iterations = iterations,
    num_topics = num_topics,
    passes = passes
)

In [288]:
from gensim.models.coherencemodel import CoherenceModel

coherencemodel = CoherenceModel(model=lda_model, texts=lyric_corpus_tokenized, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())

0.4279299874101532


In [289]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
pyLDAvis.display(vis_data)
pyLDAvis.save_html(vis_data, './Lyrics_LDA_k_'+ str(num_topics) +'.html')

c:\Users\Beste\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\Beste\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\Beste\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
c:\Users\Beste\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\Beste\AppData\Local\Programs\Python\Python312\Lib\site-pac